In [914]:
import cv2
import numpy as np

In [915]:
def display(image, scale=0.5):
    shrunk = cv2.resize(image, (0,0), fx=scale, fy=scale)
    cv2.imshow("Screen Location", shrunk)

    if cv2.waitKey(1000) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

In [916]:
billboard_colour = [150, 20, 20]
def strip_colours(image, preserve=billboard_colour):
    black_indices = np.where((image != preserve).all(axis=2))
    white_indices = np.where((image == preserve).all(axis=2))
    rtn = image.copy()
    rtn[black_indices] = [0,0,0]
    rtn[white_indices] = [255,255,255]
    return rtn

# noinspection PyDefaultArgument
def add_borders(image, border_color= [0,0,0], border_width=25):
    return cv2.copyMakeBorder(
        image,
        border_width,
        border_width,
        border_width,
        border_width,
        cv2.BORDER_CONSTANT,
        value=border_color
    )

def remove_borders(image, border_width=25):
  return image[border_width:-border_width, border_width:-border_width]

# noinspection PyShadowingNames
def get_largest_billboard_contour(image):
    edged_image = cv2.Canny(image, 30, 200)

    (contours, _) = cv2.findContours(
        edged_image.copy(),
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE
    )
    contours = sorted(
        contours,
        key=cv2.contourArea,
        reverse=True
    )[:10]


    best = (0,0)

    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        area = cv2.contourArea(contour)
        print("Found contour with {} nodes and total area {}"
              .format(len(approx), area))
        # if len(approx) == 4:
        if area > best[0]:
          best = (area, approx)

    return best[1] if best[0] > 0 else None

# noinspection PyShadowingNames
def get_contour_mask(image, contour):
    rtn = image.copy()
    cv2.drawContours(
        image=rtn,
        contours=[contour],
        contourIdx=-1,
        color=(255,255,255),
        thickness=cv2.FILLED
    )
    return rtn

def rotate(lst, x):
    return lst[-x:] + lst[:-x]
# noinspection PyShadowingNames
def get_homo_warped_decal(background, decal, contour):
    contour = [[x[0]] for x in contour]
    # contour = contour[
    contour = rotate(contour, 1)
    contour = np.float32(contour)

    h_decal, w_decal = decal.shape[:2]
    h_background, w_background = background.shape[:2]
    src = np.float32([
        [0, h_decal],
        [0,0],
        [w_decal, 0],
        [w_decal, h_decal],
    ])
    dst = np.float32(contour)
    h, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
    warped = cv2.warpPerspective(decal_image, h, (w_background, h_background))
    return warped

In [917]:
source_image = cv2.imread("../resources/new/015_00010.png")


padded_source_image = add_borders(source_image)
display(padded_source_image)

binary_image = strip_colours(padded_source_image)
display(binary_image)

In [918]:
display_image = padded_source_image.copy()
contour = get_largest_billboard_contour(binary_image)

outlined = binary_image.copy()
outlined = cv2.drawContours(
    outlined,
    [contour],
    -1,
    [0,255,0],
    3
)
display(outlined)

Found contour with 4 nodes and total area 37642.5
Found contour with 4 nodes and total area 37641.5
Found contour with 4 nodes and total area 2413.5
Found contour with 4 nodes and total area 2406.5
Found contour with 8 nodes and total area 100.5
Found contour with 8 nodes and total area 100.0
Found contour with 12 nodes and total area 16.0
Found contour with 11 nodes and total area 13.0
Found contour with 6 nodes and total area 10.0
Found contour with 8 nodes and total area 10.0


In [919]:
circles = display_image.copy()
colors = [(0,255,0),(255,0,0),(255,255,0),(0,255,255)]
for i, center in enumerate([x[0] for x in contour]):
    cv2.circle(circles, tuple(center), 20, colors[i], -1)
display(circles)

In [920]:
# decal_image = cv2.imread("../resources/new/decal_0.png")
decal_image = cv2.imread("../resources/new/milk_sip.png")

warped = get_homo_warped_decal(outlined, decal_image, contour)
display(warped)

In [921]:
blank = np.zeros(display_image.shape, dtype=np.uint8)
mask = get_contour_mask(blank, contour)
display(mask)
mask = cv2.bitwise_not(mask)
display(mask)

In [924]:
display_image_no_billboard = cv2.bitwise_and(display_image, mask)
display(display_image_no_billboard)
output_image = cv2.bitwise_or(display_image_no_billboard, warped)
display(output_image)

In [923]:
# final = cv2.bitwise_xor(display_image, display_image, mask=mask)
# display(final)